# Pymaceuticals Inc.
---

### Analysis

- The sample size for this analysis was a perfect amount, even after eliminating some duplicates we ened up with a near 50/50 split of male and female mouses. Which was great for our study creating a better representation of what the general population might look like. Looking at the clean data its still hard to understand what the data is telling us. Looking deeper using a bar and pie charts we beging to visualize the date. Capomulin, Ramicane, Infubinol, and Ceftamin were the drugs I was interested in, so I presented them in box chat where Infubinol showed a outlier highligheted in green. This meant the data set for this drug in paticular would be skewed and unreliable causing heavy influce over the data. The line plot on mouse b128 showed that the drug Capomulin showed great promise in the treatment. Due to the high drop in tumor size visulized by the steep line blue line created on the line plot. While the scatter plot showed the strong correlation between weight of the mouse vs average tumor size which can be seen by the red regression line.  
 

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "Mouse_metadata.csv"
study_results_path = "Study_results.csv"

# Read the mouse data and the study results
mouse_data = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single DataFrame
merged_df = pd.merge(study_results, mouse_data, how="left", on="Mouse ID")

# Display the new meged data frame
merged_df.head()

In [ ]:
# Checking the number of mice.
number_mice = len(merged_df["Mouse ID"].unique())
number_mice

In [ ]:
# Our data should be uniquely identified by Mouse ID and Timepoint
# Get the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
duplicated_mice = merged_df.loc[merged_df.duplicated(subset=['Mouse ID', 'Timepoint']),'Mouse ID'].unique()
duplicated_mice

In [ ]:
# Optional: Get all the data for the duplicate mouse ID. 
duplicated_df = merged_df.loc[merged_df["Mouse ID"] == "g989", :]
duplicated_df

In [ ]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
clean_data = merged_df[merged_df['Mouse ID'].isin(duplicated_mice)==False]
clean_data.head()

In [ ]:
# Checking the number of mice in the clean DataFrame.
mice = clean_data["Mouse ID"].nunique()
mice

## Summary Statistics

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary DataFrame.
mean = clean_data.groupby('Drug Regimen')['Tumor Volume (mm3)'].mean()
median = clean_data.groupby('Drug Regimen')['Tumor Volume (mm3)'].median()
variance = clean_data.groupby('Drug Regimen')['Tumor Volume (mm3)'].var()
stdev = clean_data.groupby('Drug Regimen')['Tumor Volume (mm3)'].std()
sem = clean_data.groupby('Drug Regimen')['Tumor Volume (mm3)'].sem()
summary_table = pd.DataFrame({"Mean Tumor Volume":mean,
                              "Median Tumor Volume":median,
                              "Tumor Volume Variance":variance,
                              "Tumor Volume Std. Dev.":stdev,
                              "Tumor Volume Std. Err.":sem})
summary_table

In [ ]:
# A more advanced method to generate a summary statistics table of mean, median, variance, standard deviation,
# and SEM of the tumor volume for each regimen (only one method is required in the solution)

# Using the aggregation method, produce the same summary statistics in a single line
clean_data.groupby(['Drug Regimen'])[['Tumor Volume (mm3)']].agg(['mean', 'median', 'var', 'std', 'sem'])


## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of rows (Mouse ID/Timepoints) for each drug regimen using Pandas.
mice_per_drug = clean_data["Drug Regimen"].value_counts()
plot_mouse = mice_per_drug.plot.bar(color='g')
plt.xlabel("Drug Regimen")
plt.ylabel("# of Observed Mouse Timepoints")
plt.title("Number of Mice per Treatment")

In [ ]:
# Generate a bar plot showing the total number of rows (Mouse ID/Timepoints) for each drug regimen using pyplot.
counts = clean_data['Drug Regimen'].value_counts()
counts.plot(kind="bar", color= "grey")
plt.xlabel("Drug Regimen")
plt.xticks(rotation=90)
plt.ylabel("# of Observed Mouse Timepoints")
plt.title("Number of Mice Per Treatment")
plt.show()

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using Pandas
mf_counts = clean_data.Sex.value_counts()
mf_counts.plot(kind="pie",autopct='%.1f%%')
mf_counts, plt.show()


In [ ]:
labels = ['Male', 'Female']
sizes = [50.9574468, 49.0425532]
plt.pie(sizes, labels=labels, autopct="%1.1f%%")
plt.ylabel('Sex')
plt.show

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot
#labels = ['Female', 'Male']
#sizes = mf_counts
#plot = sizes.plt.pie(y='Total Count', autopct="%1.1f%%")
#plt.title('Male vs Female Mouse Population')
#plt.ylabel('Sex')
#plt.show()

## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Start by getting the last (greatest) timepoint for each mouse
# Merge this group df with the original DataFrame to get the tumor volume at the last timepoint
last_tumor = clean_data.groupby(["Mouse ID"])['Timepoint'].max()
last_tumor = last_tumor.reset_index()
merged_data_with_result = last_tumor.merge(clean_data, on=['Mouse ID','Timepoint'], how="left")


In [ ]:
# Capomulin, Ramicane, Infubinol, and Ceftamin
# Put treatments into a list for for loop (and later for plot labels)
treatments = ["Capomulin", "Ramicane", "Infubinol", "Ceftamin"]

# Create empty list to fill with tumor vol data (for plotting)
tumor_vol = []

# Calculate the IQR and quantitatively determine if there are any potential outliers. 
for drug in treatments:
    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    final_vol = merged_data_with_result.loc[merged_data_with_result["Drug Regimen"] == drug, 'Tumor Volume (mm3)']
    
    # add subset 
    tumor_vol.append(final_vol)
    
    # Determine outliers using upper and lower bounds
    quartiles = final_vol.quantile([.25,.5,.75])
    lowerq = quartiles[0.25]
    upperq = quartiles[0.75]
    iqr = upperq-lowerq
    lower_bound = lowerq - (1.5*iqr)
    upper_bound = upperq + (1.5*iqr)
    outliers = final_vol.loc[(final_vol < lower_bound) | (final_vol > upper_bound)]
    print(f"{drug} has these potential outliers: {outliers}")

In [ ]:
# Generate a box plot that shows the distrubution of the tumor volume for each treatment group.
outlier = dict(markerfacecolor='green',markersize=15)
plt.boxplot(tumor_vol, labels = treatments, flierprops=outlier)
plt.ylabel('Final Tumor Volume (mm3)')
plt.show()

## Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a single mouse treated with Capomulin
capomulin_line = clean_data.loc[clean_data['Drug Regimen'] == "Capomulin"]
mousedata = capomulin_line.loc[capomulin_line['Mouse ID']== 'b128']
plt.plot(mousedata['Timepoint'],mousedata['Tumor Volume (mm3)'])
plt.xlabel('Timepoint (days)')
plt.ylabel('Tumor Volume (mm3)')
plt.title('Capomulin treatment of mouse b128')
plt.show()

In [ ]:
# Generate a scatter plot of mouse weight vs. the average observed tumor volume for the entire Capomulin regimen
capomulin_scatter = clean_data.loc[clean_data['Drug Regimen'] == "Capomulin"]
capomulin_average = capomulin_scatter.groupby(['Mouse ID'])[['Weight (g)', 'Tumor Volume (mm3)']].mean()
plt.scatter(capomulin_average['Weight (g)'],capomulin_average['Tumor Volume (mm3)'],color="red")
plt.xlabel('Mouse Weight (g)')
plt.ylabel('Average Tumor Volume (mm3)')
plt.title('Capomulin Regimen')
plt.show()

## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and a linear regression model 
# for mouse weight and average observed tumor volume for the entire Capomulin regimen
corr=round(st.pearsonr(capomulin_average['Weight (g)'],capomulin_average['Tumor Volume (mm3)'])[0],2)
print(f"The correlation between mouse weight and the average tumor volume is {corr}")
model = st.linregress(capomulin_average['Weight (g)'],capomulin_average['Tumor Volume (mm3)'])

y_values = capomulin_average['Weight (g)']*model[0]+model[1]
plt.scatter(capomulin_average['Weight (g)'],capomulin_average['Tumor Volume (mm3)'], color="red")
plt.plot(capomulin_average['Weight (g)'],y_values,color="Green")
plt.xlabel('Mpuse Weight (g)')
plt.ylabel('Average Tumor Volume (mm3)')
plt.show()